# Import Libraries

In [1]:
import pandas as pd
import seaborn as sns
from pathlib import Path as path
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.impute import KNNImputer

import warnings
warnings.filterwarnings('ignore')

# Import Data

In [2]:
Xtrain = pd.read_csv(path('data/Xtrain.csv'))
Ytrain = pd.read_csv(path('data/Ytrain.csv'))
Xtest = pd.read_csv(path('data/Xtest.csv'))
Ytrain = Ytrain.loc[:, ~Ytrain.columns.str.contains('^Unnamed')]
TrainData = Xtrain
TrainData = TrainData.merge(Ytrain, left_index= True, right_index= True)

# Data Cleaning

In [3]:
def cleaning(dirty_df):
    
    # Add month, day, and weekday
    dirty_df['date'] = pd.to_datetime(dirty_df['date'], format='%Y/%m/%d')
    weekday = dirty_df['date'].dt.weekday
    dirty_df.insert(1,'weekday',weekday)
    month = dirty_df['date'].dt.month
    dirty_df.insert(2,'month',month)
    
    # Add lines
    line = []
    for i in dirty_df["train"]:
        if i <= 11:
            line.append(0)
        elif i <= 21:
            line.append(1)
        elif i <= 27:
            line.append(2)
        elif i == 28:
            line.append(3)
        elif i <= 33:
            line.append(4)
        elif i <= 44:
            line.append(5)
        elif i <= 50:
            line.append(6)
        else:
            line.append(7)

    dirty_df.insert(1,'line',line)
    
    # P*q0
    dirty_df.loc[(dirty_df['p1q0'].isna()) & (dirty_df['p2q0'].isna()) & (dirty_df['p3q0'].isna()), ['p1q0','p2q0','p3q0']] = dirty_df.fillna(dirty_df.mean())
    dirty_df.loc[(dirty_df['p1q0'].isna() == False) & (dirty_df['p2q0'].isna()) & (dirty_df['p3q0'].isna()), ['p2q0','p3q0']] = dirty_df.fillna(dirty_df.mean())
    for i, row in dirty_df.iterrows():
        if math.isnan(row['p3q0']):
            if row['p1q0'] > row['p2q0']:
                diff = row['p1q0'] - (row['p1q0'] - row['p2q0'])
            else:
                diff = row['p2q0'] - (row['p2q0'] - row['p1q0'])
            dirty_df.at[i,'p3q0'] = diff
    # p0Q*
    dirty_df.loc[(dirty_df['p0q1'].isna()) & (dirty_df['p0q2'].isna()) & (dirty_df['p0q3'].isna()), ['p0q1','p0q2','p0q3']] = dirty_df.fillna(dirty_df.mean())
    dirty_df.loc[(dirty_df['p0q1'].isna() == False) & (dirty_df['p0q2'].isna()) & (dirty_df['p0q3'].isna()), ['p0q2','p0q3']] = dirty_df.fillna(dirty_df.mean())
    for i, row in dirty_df.iterrows():
        if math.isnan(row['p0q3']):
            if row['p0q1'] > row['p0q2']:
                diff = row['p0q1'] - (row['p0q1'] - row['p0q2'])
            else:
                diff = row['p0q2'] - (row['p0q2'] - row['p0q1'])
            dirty_df.at[i,'p0q3'] = diff
            
    # Drop values
    dirty_df = dirty_df.drop('date', axis=1)
    dirty_df = dirty_df.drop('hour', axis=1)
    dirty_df = dirty_df.drop('way', axis=1)
    dirty_df = dirty_df.drop('composition', axis=1)
    dirty_df = dirty_df.drop('station', axis=1)

    return dirty_df

In [4]:
TrainData = cleaning(TrainData)
TrainData

,line,weekday,month,train,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3,p0q0
0,0,0,1,1,0.234769,0.251392,0.316867,0.201,0.138,0.091,0.216
1,0,1,1,1,0.234769,0.251392,0.316867,0.204,0.152,0.106,0.216
2,0,3,1,1,0.234769,0.251392,0.316867,0.213,0.153,0.111,0.227
3,0,4,1,1,0.234769,0.251392,0.316867,0.213,0.152,0.108,0.229
4,0,0,1,1,0.234769,0.251392,0.316867,0.210,0.147,0.096,0.225
...,...,...,...,...,...,...,...,...,...,...,...
31114,0,0,5,9,0.152000,0.188600,0.157000,0.080,0.100,0.080,0.111
31115,0,1,5,9,0.153000,0.180400,0.191000,0.089,0.121,0.089,0.143
31116,0,2,5,9,0.166000,0.149000,0.168000,0.099,0.129,0.099,0.139
31117,0,3,3,9,0.182000,0.193000,0.162000,0.074,0.101,0.074,0.117


In [5]:
Xtest = cleaning(Xtest)
Xtest

,line,weekday,month,train,p1q0,p2q0,p3q0,p0q1,p0q2,p0q3
0,0,1,5,1,0.212763,0.225824,0.281484,0.226,0.165,0.128
1,0,2,5,1,0.212763,0.225824,0.281484,0.221,0.159,0.114
2,0,0,5,1,0.212763,0.225824,0.281484,0.227,0.175,0.133
3,0,2,5,1,0.212763,0.225824,0.281484,0.200,0.148,0.105
4,0,0,6,1,0.212763,0.225824,0.281484,0.221,0.169,0.126
...,...,...,...,...,...,...,...,...,...,...
13747,0,2,7,9,0.121000,0.145000,0.157150,0.071,0.095,0.071
13748,0,3,7,9,0.144000,0.139000,0.153000,0.079,0.102,0.079
13749,0,4,7,9,0.128000,0.137000,0.127000,0.072,0.089,0.072
13750,0,1,5,9,0.164000,0.187000,0.155000,0.100,0.130,0.100


# Models

In [6]:
def build_model(drop_columns):
    model = TrainData.copy()
    model = model.drop(columns=drop_columns, axis= 1)
    return model

In [7]:
def dropXTest(drop_columns):
    model = Xtest.copy()
    model = model.drop(columns=drop_columns, axis= 1)
    return model

In [8]:
def train_lr(model, export):
    Xtrain = model.drop(columns=['p0q0'], axis=1)
    Ytrain = model['p0q0']
    
    DroppedXTest = dropXTest(droppedColumns)

    x_train, x_test, y_train, y_test = train_test_split(Xtrain, Ytrain, test_size = 0.2, random_state = 42)
    LR = LinearRegression()
    LR.fit(x_train,y_train)
    
    if (export):
        y_prediction = LR.predict(DroppedXTest)
        exportToCSV(y_prediction)
        
    y_prediction =  LR.predict(x_test)
    
    score = r2_score(y_test,y_prediction)
    mse = mean_squared_error(y_test, y_prediction)

    print(f'r2 socre is:            {round(score,2)}    ->  %{round(score*100,2)}')
    print(f'Mean Squard Error is:     {round(mse,2)}    ->  %{round(mse*100,2)}')

In [9]:
def train_dnn(model, epochs, export):
    Xtrain = model.drop(columns=['p0q0'], axis=1)
    Ytrain = model['p0q0']
    
    DroppedXTest = dropXTest(droppedColumns)
    
    x_train, x_test, y_train, y_test = train_test_split(Xtrain, Ytrain, test_size = 0.2, random_state = 42)
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    model=keras.models.Sequential([
        keras.layers.Dense(512, input_dim = x_train.shape[1], activation='relu'),  
        keras.layers.Dense(512, input_dim = x_train.shape[1], activation='relu'),  
        keras.layers.Dense(units=256,activation='relu'),  
        keras.layers.Dense(units=256,activation='relu'),    
        keras.layers.Dense(units=128,activation='relu'),
        keras.layers.Dense(units=1, activation="linear"),
    ],name="Initial_model",)
    
    optimizer = keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    checkpoint_name = 'Weights\checkpoint.hdf5' 
    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_weights_only = True, save_best_only = True, mode ='auto')
    callbacks_list = [checkpoint]

    history = model.fit(x_train, y_train,
                    epochs=epochs, batch_size=1024,
                    validation_data=(x_test, y_test), 
                    callbacks=callbacks_list, 
                    verbose=1)
    
    model.load_weights('Weights\checkpoint.hdf5')
    
    if (export):
        y_prediction = model.predict(DroppedXTest)
        exportToCSV(y_prediction)
        
    y_prediction = model.predict(x_test)  
    
    score = r2_score(y_test,y_prediction)
    mse = mean_squared_error(y_test, y_prediction)

    print(f'r2 socre is:            {round(score,2)}    ->  %{round(score*100,2)}')
    print(f'Mean Squard Error is:     {round(mse,2)}    ->  %{round(mse*100,2)}')

In [10]:
def exportToCSV(data):
    data = pd.DataFrame(data).copy()
    data.rename(columns = {0:'p0q0'}, inplace = True)
    data.index = np.arange(1, len(data) + 1)
    data.to_csv("y_prediction.csv", float_format='%.3f')

# Linear Regression

In [16]:
droppedColumns = ['line', 'weekday', 'month', 'train']
model = build_model(droppedColumns)
train_lr(model, True)

r2 socre is:            0.75    ->  %75.13
Mean Squard Error is:     0.01    ->  %0.62


# Deep Neural Network

In [15]:
droppedColumns = ['line', 'weekday', 'month', 'train']
model = build_model(droppedColumns)
train_dnn(model, 250, True)

Epoch 1/250
24/25 [===========================>..] - ETA: 0s - loss: 0.0156
Epoch 1: val_loss improved from inf to 0.00657, saving model to Weights\checkpoint.hdf5
25/25 [==============================] - 2s 52ms/step - loss: 0.0155 - val_loss: 0.0066
Epoch 2/250
23/25 [==========================>...] - ETA: 0s - loss: 0.0051
Epoch 2: val_loss improved from 0.00657 to 0.00451, saving model to Weights\checkpoint.hdf5
25/25 [==============================] - 1s 32ms/step - loss: 0.0051 - val_loss: 0.0045
Epoch 3/250
25/25 [==============================] - ETA: 0s - loss: 0.0039
Epoch 3: val_loss improved from 0.00451 to 0.00363, saving model to Weights\checkpoint.hdf5
25/25 [==============================] - 1s 35ms/step - loss: 0.0039 - val_loss: 0.0036
Epoch 4/250
24/25 [===========================>..] - ETA: 0s - loss: 0.0032
Epoch 4: val_loss improved from 0.00363 to 0.00312, saving model to Weights\checkpoint.hdf5
25/25 [==============================] - 1s 32ms/step - loss: 0.0032

25/25 [==============================] - 1s 36ms/step - loss: 6.2985e-04 - val_loss: 7.4485e-04
Epoch 35/250
23/25 [==========================>...] - ETA: 0s - loss: 5.9973e-04
Epoch 35: val_loss did not improve from 0.00074
25/25 [==============================] - 1s 32ms/step - loss: 6.0190e-04 - val_loss: 8.2957e-04
Epoch 36/250
23/25 [==========================>...] - ETA: 0s - loss: 7.0141e-04
Epoch 36: val_loss did not improve from 0.00074
25/25 [==============================] - 1s 32ms/step - loss: 7.1879e-04 - val_loss: 0.0011
Epoch 37/250
23/25 [==========================>...] - ETA: 0s - loss: 7.8807e-04
Epoch 37: val_loss did not improve from 0.00074
25/25 [==============================] - 1s 29ms/step - loss: 7.7840e-04 - val_loss: 7.8848e-04
Epoch 38/250
23/25 [==========================>...] - ETA: 0s - loss: 6.5270e-04
Epoch 38: val_loss did not improve from 0.00074
25/25 [==============================] - 1s 30ms/step - loss: 6.4713e-04 - val_loss: 7.5083e-04
Epoch 39

25/25 [==============================] - ETA: 0s - loss: 4.9707e-04
Epoch 70: val_loss improved from 0.00068 to 0.00067, saving model to Weights\checkpoint.hdf5
25/25 [==============================] - 1s 32ms/step - loss: 4.9707e-04 - val_loss: 6.7059e-04
Epoch 71/250
23/25 [==========================>...] - ETA: 0s - loss: 5.1481e-04
Epoch 71: val_loss did not improve from 0.00067
25/25 [==============================] - 1s 33ms/step - loss: 5.1255e-04 - val_loss: 7.0823e-04
Epoch 72/250
24/25 [===========================>..] - ETA: 0s - loss: 5.2808e-04
Epoch 72: val_loss did not improve from 0.00067
25/25 [==============================] - 1s 30ms/step - loss: 5.2795e-04 - val_loss: 6.7967e-04
Epoch 73/250
24/25 [===========================>..] - ETA: 0s - loss: 5.1303e-04
Epoch 73: val_loss did not improve from 0.00067
25/25 [==============================] - 1s 30ms/step - loss: 5.1503e-04 - val_loss: 7.4965e-04
Epoch 74/250
24/25 [===========================>..] - ETA: 0s - loss

Epoch 106/250
24/25 [===========================>..] - ETA: 0s - loss: 4.4119e-04
Epoch 106: val_loss did not improve from 0.00065
25/25 [==============================] - 1s 30ms/step - loss: 4.4123e-04 - val_loss: 7.9076e-04
Epoch 107/250
24/25 [===========================>..] - ETA: 0s - loss: 4.6834e-04
Epoch 107: val_loss did not improve from 0.00065
25/25 [==============================] - 1s 30ms/step - loss: 4.6947e-04 - val_loss: 6.7387e-04
Epoch 108/250
24/25 [===========================>..] - ETA: 0s - loss: 4.6068e-04
Epoch 108: val_loss did not improve from 0.00065
25/25 [==============================] - 1s 30ms/step - loss: 4.6133e-04 - val_loss: 7.8265e-04
Epoch 109/250
24/25 [===========================>..] - ETA: 0s - loss: 4.5070e-04
Epoch 109: val_loss did not improve from 0.00065
25/25 [==============================] - 1s 30ms/step - loss: 4.5331e-04 - val_loss: 6.9879e-04
Epoch 110/250
25/25 [==============================] - ETA: 0s - loss: 4.2803e-04
Epoch 110:

25/25 [==============================] - 1s 34ms/step - loss: 3.9769e-04 - val_loss: 6.0233e-04
Epoch 142/250
23/25 [==========================>...] - ETA: 0s - loss: 3.7234e-04
Epoch 142: val_loss did not improve from 0.00060
25/25 [==============================] - 1s 32ms/step - loss: 3.7495e-04 - val_loss: 6.1492e-04
Epoch 143/250
23/25 [==========================>...] - ETA: 0s - loss: 3.9023e-04
Epoch 143: val_loss did not improve from 0.00060
25/25 [==============================] - 1s 31ms/step - loss: 3.9107e-04 - val_loss: 6.8849e-04
Epoch 144/250
24/25 [===========================>..] - ETA: 0s - loss: 4.1541e-04
Epoch 144: val_loss did not improve from 0.00060
25/25 [==============================] - 1s 31ms/step - loss: 4.1359e-04 - val_loss: 6.6529e-04
Epoch 145/250
23/25 [==========================>...] - ETA: 0s - loss: 3.9918e-04
Epoch 145: val_loss did not improve from 0.00060
25/25 [==============================] - 1s 30ms/step - loss: 3.9750e-04 - val_loss: 6.6030e

25/25 [==============================] - 1s 30ms/step - loss: 3.8293e-04 - val_loss: 6.9688e-04
Epoch 178/250
23/25 [==========================>...] - ETA: 0s - loss: 3.6955e-04
Epoch 178: val_loss did not improve from 0.00058
25/25 [==============================] - 1s 30ms/step - loss: 3.6793e-04 - val_loss: 5.8430e-04
Epoch 179/250
24/25 [===========================>..] - ETA: 0s - loss: 3.4559e-04
Epoch 179: val_loss did not improve from 0.00058
25/25 [==============================] - 1s 31ms/step - loss: 3.4410e-04 - val_loss: 6.3690e-04
Epoch 180/250
24/25 [===========================>..] - ETA: 0s - loss: 3.4616e-04
Epoch 180: val_loss improved from 0.00058 to 0.00057, saving model to Weights\checkpoint.hdf5
25/25 [==============================] - 1s 32ms/step - loss: 3.4559e-04 - val_loss: 5.7136e-04
Epoch 181/250
23/25 [==========================>...] - ETA: 0s - loss: 3.5332e-04
Epoch 181: val_loss did not improve from 0.00057
25/25 [==============================] - 1s 31m

25/25 [==============================] - 1s 31ms/step - loss: 3.2649e-04 - val_loss: 5.9995e-04
Epoch 214/250
24/25 [===========================>..] - ETA: 0s - loss: 3.1704e-04
Epoch 214: val_loss did not improve from 0.00057
25/25 [==============================] - 1s 32ms/step - loss: 3.1684e-04 - val_loss: 5.9085e-04
Epoch 215/250
24/25 [===========================>..] - ETA: 0s - loss: 3.2033e-04
Epoch 215: val_loss did not improve from 0.00057
25/25 [==============================] - 1s 31ms/step - loss: 3.2095e-04 - val_loss: 6.8564e-04
Epoch 216/250
25/25 [==============================] - ETA: 0s - loss: 3.6360e-04
Epoch 216: val_loss did not improve from 0.00057
25/25 [==============================] - 1s 31ms/step - loss: 3.6360e-04 - val_loss: 5.8857e-04
Epoch 217/250
24/25 [===========================>..] - ETA: 0s - loss: 3.2306e-04
Epoch 217: val_loss did not improve from 0.00057
25/25 [==============================] - 1s 32ms/step - loss: 3.2247e-04 - val_loss: 6.1526e

Epoch 250/250
23/25 [==========================>...] - ETA: 0s - loss: 3.1310e-04
Epoch 250: val_loss did not improve from 0.00057
25/25 [==============================] - 1s 30ms/step - loss: 3.1414e-04 - val_loss: 6.2269e-04
r2 socre is:            0.98    ->  %97.7
Mean Squard Error is:     0.0    ->  %0.06
